https://www.kaggle.com/c/titanic/data

In [1]:
import pandas as pd

# Load Data

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Data pre-processing

In [14]:
data_with_string_values = data.select_dtypes('object')# string
data_with_string_values.describe()

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Hamalainen, Mrs. William (Anna)",male,347082,B96 B98,S
freq,1,577,7,4,644


Sex, Cabin and Embarked columns seems to have less unique values so they can be of type catergory

In [22]:
for i in ['Sex', 'Cabin','Embarked']:
    data[i]= data[i].astype('category')

In [25]:
data.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Name             object
Sex            category
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin          category
Embarked       category
dtype: object

In [58]:
# Convert categorical variable to number
categorical_columns = data.select_dtypes('category').columns# string
for i in categorical_columns:
    data[i] = data[i].cat.codes
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,-1,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,81,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,-1,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,55,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,-1,2


In [69]:
data = data.fillna(data.mean())

# Train/Test Split

In [70]:
from sklearn.model_selection import train_test_split

In [71]:
columns_for_training = data.select_dtypes(exclude='object')
columns_for_training = columns_for_training[[i for i in columns_for_training.columns.values if i not in ['PassengerId','Survived']]]
columns_for_training.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,-1,2
1,1,0,38.0,1,0,71.2833,81,0
2,3,0,26.0,0,0,7.9250,-1,2
3,1,0,35.0,1,0,53.1000,55,2
4,3,1,35.0,0,0,8.0500,-1,2


In [72]:
X = columns_for_training

In [73]:
y = data['Survived']

In [74]:
train_x, test_x, train_y, test_y = train_test_split(X,y, train_size=0.8)

/anaconda3/envs/data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [75]:
train_y.head()

454    0
727    1
790    0
595    0
531    0
Name: Survived, dtype: int64

# Building Classifier

In [76]:
from sklearn.tree import DecisionTreeClassifier

In [77]:
tree = DecisionTreeClassifier()

In [78]:
tree.fit(train_x,train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [79]:
# Visualization

In [114]:
from sklearn.tree import export_graphviz
import graphviz

In [119]:
dot_data = export_graphviz(tree, out_file=None, feature_names=X.columns.values,class_names=['Survived','Not'],filled=True, rounded=True,  
                         special_characters=True)

In [120]:
graph = graphviz.Source(dot_data) 

In [ ]:
# run to print tree
graph

In [122]:
tree.predict(test_x)

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1])

In [2]:
from sklearn.model_selection import cross_val_score

In [124]:
cross_val_score(tree,test_x,test_y,cv=10)

array([0.68421053, 0.63157895, 0.63157895, 0.84210526, 0.83333333,
       0.82352941, 0.70588235, 0.70588235, 0.76470588, 0.70588235])

# Preparing Output File

In [147]:
submission_data=pd.read_csv('test.csv')
for i in ['Sex', 'Cabin','Embarked']:
    submission_data[i]= submission_data[i].astype('category')
categorical_columns = submission_data.select_dtypes('category').columns# string
for i in categorical_columns:
    submission_data[i] = submission_data[i].cat.codes
    
passenger_ids = submission_data['PassengerId']
submission_data.fillna(submission_data.mean())
submission_data = submission_data[[i for i in columns_for_training.columns.values if i not in ['PassengerId','Survived']]]
submission_data = submission_data.fillna(submission_data.mean())
submission_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,34.5,0,0,7.8292,-1,1
1,3,0,47.0,1,0,7.0000,-1,2
2,2,1,62.0,0,0,9.6875,-1,1
3,3,1,27.0,0,0,8.6625,-1,2
4,3,0,22.0,1,1,12.2875,-1,2


In [148]:
prediction = tree.predict(submission_data)
prediction

array([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [154]:
df = pd.DataFrame({'PassengerId':passenger_ids,'Survived':prediction})
df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,1


In [156]:
df.to_csv('sklearn_decision_tree_submission.csv', index=False)